In [1]:
import sys
sys.executable

'd:\\Code-Projects\\University\\Fall2023\\Grad\\LipNet-Assistive-AI\\.conda\\python.exe'

In [2]:
WORK_DIR= r"E:/Video Links Dataset"
TEMP_VIDEOS = r"E:/Video Links Dataset/temp_videos"
VIDEOS = r"E:/Video Links Dataset/videos"
ALIGNMENTS = r"E:/Video Links Dataset/alignments"
TEMP_ALIGNMENTS = r"E:/Video Links Dataset/temp_alignments"

In [3]:
from pytube import YouTube
from moviepy.editor import VideoFileClip
from moviepy.video.fx.all import crop
import os
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
from time import sleep
import pysrt
import re
import string
import subprocess
import glob
import shutil


pygame 2.5.2 (SDL 2.28.3, Python 3.9.18)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
# Function to download video
def download_video(url, id):
        try:
            yt = YouTube(url)

            stream = (
                yt.streams.filter(progressive=True, file_extension="mp4").get_highest_resolution()
            )

            if "a.ar" in yt.captions.keys():
                captions = yt.captions.get_by_language_code("a.ar")
                captions.download(
                    srt=True,
                    output_path=TEMP_ALIGNMENTS,
                    title=f"{id}")

                stream.download(
                    filename=fr"{TEMP_VIDEOS}/{id}.mp4"
                )

                return fr"{TEMP_VIDEOS}/{id}.mp4",fr"{TEMP_ALIGNMENTS}/{id} (a.ar).srt", id

            else:
                print("skipping video no arabic subtitles")
                return False, False, False
            
        except Exception as e:
             print(e)
             return None,None, None
        
def check_outliers(numbers):
    if not numbers:
        return False  # Return False if the list is empty

    mean = sum(numbers) / len(numbers)
    
    # Calculate standard deviation
    variance = sum((x - mean) ** 2 for x in numbers) / len(numbers)
    std_dev = variance ** 0.5
    
    # Define thresholds
    lower_threshold = mean - 3 * std_dev
    upper_threshold = mean + 3 * std_dev
    
    # Check if any number falls outside the thresholds
    for num in numbers:
        if num < lower_threshold or num > upper_threshold:
            return True  # Return True if any number is outside the thresholds
    
    return False  # Return False if no number is outside the thresholds

def calculate_frame_differences(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Couldn't open video file.")
        return None
    
    # Read the first frame
    ret, prev_frame = cap.read()
    if not ret:
        print("Error: Couldn't read the first frame.")
        return None
    
    # Initialize list to store differences
    differences = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Calculate absolute difference between frames
        diff = cv2.absdiff(prev_frame, frame)
        
        # Convert difference image to grayscale
        gray_diff = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
        
        # Calculate the average pixel difference
        avg_diff = np.mean(gray_diff)
        differences.append(avg_diff)
        
        # Update the previous frame
        prev_frame = frame.copy()
    cap.release()

    return(check_outliers(differences))

def english_to_arabic_numbers(text):

    numbers_dict = {
        '0': '٠',
        '1': '١',
        '2': '٢',
        '3': '٣',
        '4': '٤',
        '5': '٥',
        '6': '٦',
        '7': '٧',
        '8': '٨',
        '9': '٩',
    }
    for eng_num, arabic_num in numbers_dict.items():
        text = text.replace(eng_num, arabic_num)
    return text

def remove_special_char(text):
    for i in string.punctuation:
        text = text.replace(i, " ")

    return text

def remove_repeating_spaces(text):
    cleaned_text = re.sub(r' +', ' ', text)
    return cleaned_text

def check_if_not_arabic(text):
    ar = "ابتثجحخدذرزسشصضطظعغفقكلمنهويىء٠١٢٣٤٥٦٧٨٩ "
    for i in text:
        if i not in ar:
            return True
    return False

def filter_srt_sub(text):
    
    if(len(text) < 2):
        return False

    if(text == "[موسيقى]"):
        return False
    
    text = english_to_arabic_numbers(text)
    text = remove_special_char(text)
    text = remove_repeating_spaces(text)
    
    if(check_if_not_arabic(text)):
        return False
    
    if(len(text) < 2):
        return False
    
    return text


def cut_from_srt_file(video_path, srt_path, output_dir_videos,output_dir_str, id):
    
    subs = pysrt.open(srt_path)
    video = VideoFileClip(video_path)

    os.makedirs(output_dir_videos, exist_ok=True)

    for idx, sub in enumerate(subs):
        try:
            text = filter_srt_sub(sub.text.strip())

            if( not text):
                continue

            start_time = sub.start.ordinal /1000.0
            end_time = sub.end.ordinal / 1000.0     

            video_part = video.subclip(start_time, end_time)
            output_path = os.path.join(output_dir_videos, f"{id}_{idx+1}.mp4")
            
            with open(output_dir_str+f"/{id}_{idx+1}.align","w") as writer:
                writer.write(text.replace(" ","\n"))

            video_part.write_videofile(output_path,verbose=False, progress_bar=False) 
        except:
            video.close()
            pass
    video.close()

def video_is_accpeted(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return None
    
    mp_face_mesh = mp.solutions.face_mesh
    mp_face_detection = mp.solutions.face_detection

    video_fps = int(cap.get(cv2.CAP_PROP_FPS))
    video_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    video_duration = video_frames // video_fps


    if (int(video_duration) != 3):

        return False
    if((video_fps < 25) or (video_fps > 60)):

        return False


    lip_pairs = [[13, 14], [82, 87], [312, 317],[81,178],[311,402],[310,318],[80,88]]
    with mp_face_detection.FaceDetection(
        model_selection=2, min_detection_confidence=1
    ) as face_detection:
        with mp_face_mesh.FaceMesh(min_detection_confidence=0.5) as face_mesh:
            orientation_list=[]
            diffs = []
            # loop over all frames in the video
            for _ in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
                ret, frame = cap.read()

                # convert to rgb
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                # get the keypoints (x,y)s of each point of a face in the frame
                results = face_detection.process(frame)
                if results.detections:
                    if(len(results.detections) !=1):
                        print("1")
                        return False
                else:
                    print("2")

                    return False
                
                # check orientation
                # check if lips closed

                face_mesh_result = face_mesh.process(frame)
                if face_mesh_result.multi_face_landmarks:
                    for face_landmarks in face_mesh_result.multi_face_landmarks:

                        landmarks = face_landmarks.landmark
                        right_x = int(landmarks[454].x * frame.shape[1])
                        left_x = int(landmarks[234].x * frame.shape[1])
                        upper_x = int(landmarks[10].x * frame.shape[1])
                        lower_x = int(landmarks[152].x * frame.shape[1])

                        center_x_left = (upper_x + lower_x)//2
                        center_x_right = (upper_x + lower_x)//2

                        if(left_x>center_x_left):
                            orientation_list.append(0)
                        elif(right_x<center_x_right):
                            orientation_list.append(0)
                        else:
                            orientation_list.append(1)
                            
                        diff = 0

                        for lip_pair in lip_pairs:
                            upper_lip_x, upper_lip_y = int(
                                landmarks[lip_pair[0]].x * frame.shape[1]
                            ), int(landmarks[lip_pair[0]].y * frame.shape[0])
                            lower_lip_x, lower_lip_y = int(
                                landmarks[lip_pair[1]].x * frame.shape[1]
                            ), int(landmarks[lip_pair[1]].y * frame.shape[0])
                            diff += ((upper_lip_x - lower_lip_x) ** 2) + (
                                (upper_lip_y - lower_lip_y) ** 2
                            )
                        diffs.append(1 if diff>50 else 0)
            if(len(orientation_list) == 0 ):
                return False
            if(len(diffs) == 0 ):
                return False
            
            if((sum(orientation_list)/len(orientation_list)) !=1.0):
                print("3")
                return False
            
            if(not (sum(diffs)/len(diffs)) > 0.3):
                print("4")

                return False
            
    cap.release()
    return not calculate_frame_differences(video_path)
    

def filter_videos(videos_path,alignments_path,output_videos,output_alignments,id):
    for i in glob.glob(videos_path+fr"/{id}_*.mp4"):
        video_path = i.replace("\\","/")
        alignment_path = alignments_path+"/"+re.split(r"\\|/",i)[-1].split(".")[0]+".align"
        output_video_path = output_videos+"/"+re.split(r"\\|/",i)[-1]
        output_alignment_path = output_alignments+"/"+re.split(r"\\|/",i)[-1].split(".")[0]+".align"

    
        if(video_is_accpeted(video_path)):
            shutil.copy(video_path, output_video_path)
            shutil.copy(alignment_path, output_alignment_path)




In [5]:
def pipline(url, name):
    fails = 10

    video, alignment, id = download_video(url,name)
    while (video == None) and (fails > 0):
        fails -= 1
        sleep(4)
        video, alignment, id = download_video(url,name)

    if not video:
        return False

    subprocess.run(["go","run",".",alignment])

    cut_from_srt_file(video,alignment+".fixed",TEMP_VIDEOS,TEMP_ALIGNMENTS, id)


    filter_videos(TEMP_VIDEOS,TEMP_ALIGNMENTS,VIDEOS,ALIGNMENTS,id)

    return True
    

In [6]:
df = pd.read_csv(fr"{WORK_DIR}/Video Links Dataset.csv")
df.head()

,link,name,id,creator,parsed
0,https://www.youtube.com/watch?v=s0pbzSC81Kc,كتابة الكلام على الفيديو بشكل تلقائي بالذكاء ا...,0,0,1
1,https://www.youtube.com/watch?v=FYqNG3zwXL4&t=...,فيلم الصفاره,1,0,0
2,https://www.youtube.com/watch?v=3r3IOzNNKRM,جت كدا اياد الموجي وساره هاني,2,0,0
3,https://www.youtube.com/watch?v=Z31F0zVuHOg&t=...,فيلم العارف,3,0,0
4,https://www.youtube.com/watch?v=0SGpG7m4xzE,فيلم الخليه,4,0,0


In [7]:
df[df.duplicated('link',keep=False)]

,link,name,id,creator,parsed


In [8]:
results = []

In [9]:
status = []

for idx,row in enumerate(df.values):

    for i in glob.glob(fr"{TEMP_VIDEOS}/*"):
        os.remove(i)
    for i in glob.glob(Fr"{TEMP_ALIGNMENTS}/*"):
        os.remove(i)

    if int(row[4]):
        continue
    else:
        for i in glob.glob(fr"{VIDEOS}/{row[2]}_*.mp4"):
            os.remove(i)
        for i in glob.glob(fr"{ALIGNMENTS}/{row[2]}_*.mp4"):
            os.remove(i)
        result = pipline(row[0], row[2])
        results.append(result)
        df["parsed"][idx] = 1
        df.to_csv(fr"{WORK_DIR}/Video Links Dataset.csv",index=False)    
        print("Parsed Video : ",row[2])


In [ ]:
# code in captions.py
    # def xml_caption_to_srt(self, xml_captions: str) -> str:
    #     """Convert xml caption tracks to "SubRip Subtitle (srt)".

    #     :param str xml_captions:
    #     XML formatted caption tracks.
    #     """
    #     segments = []
    #     root = ElementTree.fromstring(xml_captions)
    #     i=0
    #     for child in list(root.iter("body"))[0]:
    #         if child.tag == 'p':
    #             caption = ''
    #             if len(list(child))==0:
    #                 # instead of 'continue'
    #                 caption = child.text
    #             for s in list(child):
    #                 if s.tag == 's':
    #                     caption += ' ' + s.text
    #             caption = unescape(caption.replace("\n", " ").replace("  ", " "),)
    #             try:
    #                 duration = float(child.attrib["d"])/1000.0
    #             except KeyError:
    #                 duration = 0.0
    #             start = float(child.attrib["t"])/1000.0
    #             end = start + duration
    #             sequence_number = i + 1  # convert from 0-indexed to 1.
    #             line = "{seq}\n{start} --> {end}\n{text}\n".format(
    #                 seq=sequence_number,
    #                 start=self.float_to_srt_time_format(start),
    #                 end=self.float_to_srt_time_format(end),
    #                 text=caption,
    #             )
    #             segments.append(line)
    #             i += 1
    #     return "\n".join(segments).strip()